In [1]:
import numpy as np
import heapq

class Node:
    pass
    """结点"""
    def __init__(self,feature_idx,sample_val,parent=None):
        pass
        """
            feature_val 记录保存在结点中的样本的值
        """
        self.feature_idx = feature_idx
        self.sample_val = sample_val
        self.left = None
        self.right = None

class KNN:
    pass
    """
    k-nearest neighbor的kd树分类算法实现
    kd树是存储k维空间的树结构，与k邻近算法中的k意义不同
    """

    def fit(self,X,y):
        pass
        """训练的过程其实就是构造kd树的过程"""
        self.root = self._build_tree(X,0)

    def _build_tree(self,X,deepth):
        pass
        """
        对于k维空间数据集
        对深度为j的结点，选择 l = j(mod k) 坐标轴进行切分"""
        n_samples,n_features = X.shape
        if n_samples == 0:
            return None
        f_idx = deepth % n_features
        if n_samples == 1:
            return Node(feature_idx=f_idx, sample_val= X[0])

        sort_idx = np.argsort(X[:,f_idx])
        X = X[sort_idx]

        median_idx = n_samples//2
        #根据中位数划分左右子集
        X_left = X[0:median_idx]
        X_right = X[median_idx+1:n_samples]

        #结点保存划分点的值和相应的标签
        node = Node(feature_idx=f_idx,sample_val=X[median_idx])
        node.left = self._build_tree(X_left,deepth+1)
        node.right = self._build_tree(X_right,deepth+1)
        return node


    def preOrder(self,node):
        if node == None:
            return
        print(node.sample_val)
        self.preOrder(node.left)
        self.preOrder(node.right)

    def _visit(self,node,x):
        pass
        """访问结点"""
        if node != None:
            dis = x[node.feature_idx] - node.sample_val[node.feature_idx]
            if dis < 0:
                self._visit(node.left,x)
            else:
                self._visit(node.right,x)

            curr_dis = np.linalg.norm(x-node.sample_val,ord=2)
            heapq.heappushpop(self.k_nearest_neighbor,(-curr_dis,node))
            if -(self.k_nearest_neighbor[0][0]) > abs(dis):
                self._visit(node.right if dis < 0 else node.left,x)

    def _search_nearest_k_node(self,x,near_k=1):
        pass
        """这里的k是寻找离X_test 最近的k个节点，采用的是欧式距离"""
        #python的堆是最小堆，对于最大堆的支持很不友好
        #所以转换一个思路，把距离取父值，值小的说明距离大
        self.k_nearest_neighbor = [(-np.inf,None)]*near_k
        self._visit(self.root,x)
        self.k_nearest_neighbor = np.array(
            [i[1].sample_val for i in heapq.nlargest(near_k,self.k_nearest_neighbor)]
        )
        return self.k_nearest_neighbor


    def search(self,X_test,k=1):
        pass
        """这里的k是寻找离X_test 最近的k个节点"""
        res = []
        for i in range(X_test.shape[0]):
            res.append(self._search_nearest_k_node(X_test[i],near_k=k))
        return res



In [2]:
arr =np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]])

knn = KNN()
knn.fit(arr,np.arange(arr.shape[0],dtype="int"))
# print("======kd树前序遍历=========")
# knn.preOrder(knn.root)

X_test = np.array([[2,4]])
near = knn.search(X_test,k=2)

print("查找结果：",near)


查找结果： [array([[2, 3],
       [5, 4]])]


剩下的如果需要做分类则可以求结果的众数，回归则求结果的平均值即可（结点可以再存储一下标签值）